In [4]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from IPython.display import clear_output, display, HTML
import ipywidgets
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import scipy

In [5]:
dpath = Path(r'../../data/')
st_samp = np.squeeze(np.load(dpath / 'spike_times.npy'))
sc = np.squeeze(np.load(dpath / 'spike_clusters.npy'))
imIDs = np.squeeze(np.load(dpath / 'natImsIDs.npy'))
imTimes = np.squeeze(np.load(dpath / 'natImsOnTimes.npy'))

st_s = st_samp / 3e4 # convert from 30khz samples to seconds
# set up bin edges - 20 ms here
bins_s = np.arange(np.min(st_s), np.max(st_s), 0.02)
# make list of lists for spike times specific to each cluster
spikes = [st_s[sc == clu] for clu in np.unique(sc)]
# bin
spk_mat = []
for clu in spikes:
    counts, _ = np.histogram(clu, bins_s)
    spk_mat.append(counts)
spk_mat = np.array(spk_mat) # should be [#neurons, #bins]
# and your timestamps are the left bin edges, e.g. plot avg pop rate over time
# plt.plot(bins_s[:-1], np.mean(spk_mat, axis=0))

timestamps = bins_s[:-1]
ts_to_ix = scipy.interpolate.interp1d(timestamps, np.arange(len(timestamps)), kind='previous')
img_onIxs = ts_to_ix(imTimes).astype('int')

In [6]:
window = np.r_[-8:20]
tr_ts = window / 0.02

# indexing for each trial
tr_ixs = [window + i for i in img_onIxs]
tr_ixs = np.array(tr_ixs)

# pick these timepoints out of spike matrix
spks_trs = spk_mat.T[tr_ixs]

# separate by image identity
im_order = [np.squeeze(np.argwhere(imIDs == i)) for i in np.unique(imIDs)]
im_order = np.array(im_order)

spks_trs = spks_trs[im_order]

In [7]:
def plotPSTH(neuron):
    for i in range(len(np.unique(imIDs))):
        plt.plot(tr_ts, np.mean(spks_trs[i, :, :, neuron], axis=0))
    plt.axvspan(0, 300, color='gray', lw=0, alpha=0.3)
    plt.xlabel('time from stimulus onset (ms)')
    plt.ylabel('spks / bin')
    plt.title(f'neuron {neuron}, spk rate {np.mean(spk_mat[neuron])/0.02:.1f} s$^{{-1}}$')
w = ipywidgets.interactive(plotPSTH, neuron=ipywidgets.IntSlider(min=0, max=spk_mat.shape[0]-1, step=1, value=0))
display(w)

interactive(children=(IntSlider(value=0, description='neuron', max=752), Output()), _dom_classes=('widget-inte…